# TOKENIZATION METHODS

# METHOD 1: Unstructured default tokenization

# Install the packages and libmagic for automatic file type detection
pip install "unstructured[all-docs]"
pip install unstructured-client
brew install libmagic

In [1]:
from unstructured_client import UnstructuredClient
s = UnstructuredClient(api_key_auth="UXNnvEtTT7FyVI1qY1R85616zcN8eO")

In [7]:
filename = "./gpt-4.pdf"
file = open(filename, "rb")

In [9]:
from unstructured_client.models import shared

req = shared.PartitionParameters(
    # Note that this currently only supports a single file
    files=shared.Files(
        content=file.read(),
        file_name=filename,
    ),
    # Other partition params
    strategy="fast", # fast, hi_res, auto. For details see https://unstructured-io.github.io/unstructured/best_practices/strategies.html
)
print(req)

PartitionParameters(chunking_strategy=None, combine_under_n_chars=None, coordinates=None, encoding=None, files=Files(content=b'', file_name='./gpt-4.pdf'), gz_uncompressed_content_type=None, hi_res_model_name=None, include_page_breaks=None, languages=None, max_characters=None, multipage_sections=None, new_after_n_chars=None, output_format=None, pdf_infer_table_structure=None, skip_infer_table_types=None, strategy='fast', xml_keep_tags=None)


In [6]:
# Default Unstructured partitioning

from unstructured_client.models.errors import SDKError

try:
    res = s.general.partition(req)
    for i in range(10):
        print(res.elements[i])
except SDKError as e:
    print(e)

{'type': 'UncategorizedText', 'element_id': 'a04b820b51c760a41415c57c1eef8f08', 'text': '3 2 0 2', 'metadata': {'languages': ['eng'], 'page_number': 1, 'filename': 'gpt-4.pdf', 'filetype': 'application/pdf'}}
{'type': 'Title', 'element_id': '452202c3d8a420d49447943b87c30d0e', 'text': 'r a', 'metadata': {'languages': ['eng'], 'page_number': 1, 'filename': 'gpt-4.pdf', 'filetype': 'application/pdf'}}
{'type': 'UncategorizedText', 'element_id': '27aa14beb95d26e84d7cb2b9d4dbec83', 'text': 'M 7 2', 'metadata': {'languages': ['eng'], 'page_number': 1, 'parent_id': '452202c3d8a420d49447943b87c30d0e', 'filename': 'gpt-4.pdf', 'filetype': 'application/pdf'}}
{'type': 'Title', 'element_id': '3e16d5218a727bdf70d048a818de45a9', 'text': '] L C . s c [', 'metadata': {'languages': ['eng'], 'page_number': 1, 'filename': 'gpt-4.pdf', 'filetype': 'application/pdf'}}
{'type': 'UncategorizedText', 'element_id': 'fef8fa7a97cf378a4fe8d4292a267851', 'text': '8 0 5 2 1 8 4 / t i', 'metadata': {'languages': ['

# METHOD 2: Proposition-based tokenization

# Install package to read PDF
pip install PyPDF2
pip install transformers
pip install torch

In [2]:
# Proposition-based retrieval

# Read all text from PDF
import PyPDF2
reader = PyPDF2.PdfReader('gpt-4.pdf')
content = ""
for i in range(len(reader.pages)):
    content += reader.pages[i].extract_text()

# Use a small portion of text for testing
content = content[:1000]

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import json

model_name = "chentong00/propositionizer-wiki-flan-t5-large"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

input_ids = tokenizer(content, return_tensors="pt").input_ids
outputs = model.generate(input_ids.to(device), max_new_tokens=512).cpu()

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
try:
    prop_list = json.loads(output_text)
except:
    prop_list = []
    print("[ERROR] Failed to parse output text as JSON.")
print(json.dumps(prop_list, indent=2))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[
  "This technical report presents GPT-4, a large multimodal model.",
  "GPT-4 can accept image and text inputs and produce text outputs.",
  "GPT-4 is less capable than humans in many real-world scenarios.",
  "GPT-4 exhibits human-level performance on various professional and academic benchmarks.",
  "GPT-4 passed a simulated bar exam with a score around the top 10% of test takers.",
  "GPT-4 is a Transformer-based model pre-trained to predict the next token in a document.",
  "The post-training alignment process results in improved performance on measures of factuality and adherence to desired behavior.",
  "A core component of this project was developing infrastructure and optimization methods that behave predictably across a wide range of scales.",
  "This allowed us to accurately predict some aspects of GPT-4's performance based on models trained with no more than 1/1,000th the compute of GPT-4."
]
